# Data Exploration

In [26]:
# Import libraries in alphabetical order
import numpy as np
import pandas as pd


## GeoFeatures_Zurich_provided_by_UrbanDataLabs

### Fields:

- **x, y:** Coordinates in EPSG 2056

- **lat, lng:** Coordinates in latitude, longitude

- **hh_ha, pers_ha:** Density of households and persons per hectare

- **pt_class:** Public transport quality class according to Federal Office for Spatial Development

- **pt_dis:** Distance to next stop (as the crow flies)

- **station_dis:** Distance to railway station

- **noise_street:** Road noise according to BAFU

- **bldg_foot:** Building footprint in m²

- **bldg_corner:** Building is located at an intersection

- **bldg_500:** Number of buildings in 500m radius

- **net_acc:** Size of catchment area in 2 min driving time

In [27]:
# Read GeoFeatures_Zurich_provided_by_UrbanDataLabs file
df = pd.read_csv("GeoFeatures_Zurich_provided_by_UrbanDataLabs.csv")
df.head()

,x,y,geom,hh_ha,pers_ha,pt_class,pt_dis,station_dis,noise_street,bldg_foot,bldg_corner,bldg_500,net_acc,lat,lng
0,2682762.5,1246387.5,01010000200808000000000040C577444100000080B304...,8,10,1.0,55,1725,45,4204,t,1402050,1043508.0,47.3631,8.53425
1,2682762.5,1246412.5,01010000200808000000000040C577444100000080CC04...,53,100,1.0,50,1700,69,4204,t,1402050,1043508.0,47.3634,8.53425
2,2682787.5,1246337.5,010100002008080000000000C0D1774441000000808104...,8,10,1.0,75,1770,58,4204,t,1402050,1043508.0,47.3627,8.53457
3,2682787.5,1246362.5,010100002008080000000000C0D1774441000000809A04...,8,10,1.0,55,1745,56,4204,t,1402050,1043508.0,47.3629,8.53457
4,2682787.5,1246387.5,010100002008080000000000C0D177444100000080B304...,8,10,1.0,35,1720,63,4204,t,1402050,1043508.0,47.3631,8.53458


In [28]:
# Select columns
df = df[["lat", "lng", "hh_ha", "pers_ha", "pt_dis", "station_dis", "bldg_foot", "bldg_500", "net_acc"]]
df = df.rename(columns={"hh_ha": "hh_den",
                        "pers_ha": "pop_den",
                        "station_dis": "ts_dist",
                        "bldg_foot" : "building_den",
                        "bldg_500": "builing_den_500",
                        "net_acc": "catchment_area_driv"})

In [38]:
# Null values

# Summary by column
summary_nulls = df.isnull().sum()/len(df)*100
summary_nulls

lat                    0.0
lng                    0.0
hh_den                 0.0
pop_den                0.0
pt_dis                 0.0
ts_dist                0.0
building_den           0.0
builing_den_500        0.0
catchment_area_driv    0.0
migros_boolean         0.0
dtype: float64

In [29]:
df_corr = df[["hh_den", "pop_den", "pt_dis", "ts_dist", "building_den", "builing_den_500", "catchment_area_driv"]]
df_corr.describe()

,hh_den,pop_den,pt_dis,ts_dist,building_den,builing_den_500,catchment_area_driv
count,126331.00000,126331.000000,126331.000000,126331.000000,126331.000000,1.263310e+05,1.234680e+05
mean,25.42961,52.510904,208.818659,4146.849111,1745.523862,6.990749e+05,7.195469e+05
std,33.31709,66.065373,148.929058,2066.859348,4378.829480,6.052859e+05,3.801618e+05
min,0.00000,0.000000,0.000000,0.000000,23.000000,2.840000e+02,1.810000e+02
25%,0.00000,0.000000,110.000000,2615.000000,220.000000,2.593480e+05,4.180810e+05
50%,11.00000,26.000000,175.000000,3950.000000,467.000000,5.061990e+05,7.140580e+05
75%,41.00000,87.000000,255.000000,5335.000000,1220.000000,9.724245e+05,1.029945e+06
max,255.00000,546.000000,1290.000000,10230.000000,58985.000000,3.350409e+06,1.657424e+06


In [30]:
# Correlation?
df_corr.corr(method="pearson")

,hh_den,pop_den,pt_dis,ts_dist,building_den,builing_den_500,catchment_area_driv
hh_den,1.000000,0.961563,-0.276513,-0.254892,-0.069816,0.333425,0.415500
pop_den,0.961563,1.000000,-0.281431,-0.234213,-0.079519,0.303619,0.403119
pt_dis,-0.276513,-0.281431,1.000000,0.356661,-0.084605,-0.351942,-0.447405
ts_dist,-0.254892,-0.234213,0.356661,1.000000,-0.145825,-0.647647,-0.434190
building_den,-0.069816,-0.079519,-0.084605,-0.145825,1.000000,0.311981,0.044833
builing_den_500,0.333425,0.303619,-0.351942,-0.647647,0.311981,1.000000,0.599545
catchment_area_driv,0.415500,0.403119,-0.447405,-0.434190,0.044833,0.599545,1.000000


# Creating fake data

In [31]:
# Migros column (1/0)
np.random.seed(22)

df['migros_boolean'] = np.random.choice([0, 1], df.shape[0])
df.head()

,lat,lng,hh_den,pop_den,pt_dis,ts_dist,building_den,builing_den_500,catchment_area_driv,migros_boolean
0,47.3631,8.53425,8,10,55,1725,4204,1402050,1043508.0,1
1,47.3634,8.53425,53,100,50,1700,4204,1402050,1043508.0,0
2,47.3627,8.53457,8,10,75,1770,4204,1402050,1043508.0,0
3,47.3629,8.53457,8,10,55,1745,4204,1402050,1043508.0,0
4,47.3631,8.53458,8,10,35,1720,4204,1402050,1043508.0,0


In [32]:
df = df.dropna()

# Logistic Regression

In [33]:
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm


In [35]:
# Dependant variable: "migros_boolean"
dep_var = ["migros_boolean"]

# Independent variables:
indep_var = ["hh_den", "pop_den", "pt_dis", "ts_dist", "building_den", "builing_den_500", "catchment_area_driv"]

# Model:
x = df[indep_var]
y = df[dep_var]

logit_model = sm.Logit(y,x)
result = logit_model.fit()
print(result.summary2())


Optimization terminated successfully.
         Current function value: 0.693121
         Iterations 3
                          Results: Logit
Model:              Logit            Pseudo R-squared: 0.000      
Dependent Variable: migros_boolean   AIC:              171170.5076
Date:               2021-09-03 13:56 BIC:              171238.5737
No. Observations:   123468           Log-Likelihood:   -85578.    
Df Model:           6                LL-Null:          -85581.    
Df Residuals:       123461           LLR p-value:      0.40833    
Converged:          1.0000           Scale:            1.0000     
No. Iterations:     3.0000                                        
------------------------------------------------------------------
                     Coef.  Std.Err.    z    P>|z|   [0.025 0.975]
------------------------------------------------------------------
hh_den              -0.0000   0.0006 -0.0113 0.9910 -0.0012 0.0012
pop_den              0.0002   0.0003  0.7214 0.4706 -